# Bundesliga bereinigen

## Imports

In [1]:
import pandas as pd
import re

## Laden der Excel-Datei

In [2]:
file_path = '/Users/youri/VSC Data/Data Analytics/Fussballdaten/1Bundesliga_alle_tore_zeiten_15_23.xlsx'
df = pd.read_excel(file_path)

# Anzeigen der ersten paar Zeilen der Tabelle zur Überprüfung
df.head()


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw
0,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,NaN,NaN,NaN
1,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:1,NaN,NaN
2,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:0,NaN,NaN
3,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,NaN,NaN,NaN
4,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,1:0,NaN,NaN


In [3]:
def clean_data(df):
    """
    Bereinigt den DataFrame, indem Zeilen entfernt werden, in denen alle angegebenen Spalten leer sind.

    :param df: Der zu bereinigende DataFrame.
    :return: Der bereinigte DataFrame.
    """

    columns_to_check = ['final_score_raw', 'goal_raw', 'goal_time_raw', 'extratime_raw']
    # Entfernen von Zeilen, in denen alle spezifizierten Spalten leer sind
    df_cleaned = df.dropna(subset=columns_to_check, how='all')
    return df_cleaned

# Beispiel für die Verwendung der Funktion
df_cleaned = clean_data(df)
df_cleaned.head()


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw
0,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,NaN,NaN,NaN
1,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:1,NaN,NaN
2,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:0,NaN,NaN
3,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,NaN,NaN,NaN
4,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,1:0,NaN,NaN


In [4]:
# Funktionen zur Extraktion von Spieltag und Saison
def extract_spieltag(text):
    match = re.search(r'(\d+)\. Spieltag', text)
    return match.group(1) if match else None

def extract_saison(text):
    match = re.search(r'\b(\d{4}/\d{4})\b', text)
    return match.group(1) if match else None

# Test der Funktionen mit einem Beispieltext
example_text = "Bundesliga 2022/2023 - 34. Spieltag - Sa., 27...."

test_spieltag = extract_spieltag(example_text)
test_saison = extract_saison(example_text)

test_spieltag, test_saison


('34', '2022/2023')

In [5]:
# Erstellen eines DataFrame
df_example = pd.DataFrame(df_cleaned)

# Anwenden der Funktionen zur Erstellung der neuen Spalten
df_example['League'] = 'Bundesliga'
df_example['Spieltag'] = df_example['matchday_raw'].apply(extract_spieltag)
df_example['Saison'] = df_example['matchday_raw'].apply(extract_saison)

# Anzeigen der ersten paar Zeilen des aktualisierten DataFrame
df_example.head()

,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,League,Spieltag,Saison
0,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,NaN,NaN,NaN,Bundesliga,1,2014/2015
1,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:1,NaN,NaN,Bundesliga,1,2014/2015
2,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:0,NaN,NaN,Bundesliga,1,2014/2015
3,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,NaN,NaN,NaN,Bundesliga,1,2014/2015
4,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,1:0,NaN,NaN,Bundesliga,1,2014/2015


In [6]:
def merge_goal_times(df):
    """
    Fügt die Torzeiten in die gleiche Zeile wie die Tore ein, ohne Daten zu löschen, wenn es mehr Torzeiten als Tore gibt.

    :param df: Der zu bearbeitende DataFrame.
    :return: Der bearbeitete DataFrame.
    """
    # Gruppieren der Daten nach Spielen
    grouped = df.groupby(['matchday_raw', 'hometeam_raw', 'awayteam_raw'])

    # Listen zur Speicherung der bearbeiteten Daten
    merged_data = []

    # Durchlaufen jeder Gruppe (jedes Spiels)
    for _, group in grouped:
        # Extrahieren der Tore und Torzeiten
        goals = group['goal_raw'].dropna().tolist()
        times = group['goal_time_raw'].dropna().tolist()

        # Sicherstellen, dass die Länge von Toren und Zeiten gleich ist
        max_length = max(len(goals), len(times))
        goals += [None] * (max_length - len(goals))  # Füllen mit None, falls notwendig
        times += [None] * (max_length - len(times))  # Füllen mit None, falls notwendig

        # Erstellen einer neuen DataFrame für das aktuelle Spiel
        game_data = group.iloc[:max_length].copy()
        game_data['goal_raw'] = goals
        game_data['goal_time_raw'] = times

        # Hinzufügen der bearbeiteten Daten zur Liste
        merged_data.append(game_data)

    # Zusammenführen aller bearbeiteten Spiele zurück in einen DataFrame
    return pd.concat(merged_data)

# Beispiel für die Verwendung der Funktion
# df_example = pd.DataFrame(...)  # Ersetzen Sie dies durch Ihren DataFrame
df_cleaned = merge_goal_times(df_example)
df_cleaned.head(10)


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,League,Spieltag,Saison
0,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:1,52',NaN,Bundesliga,1,2014/2015
1,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:0,47',NaN,Bundesliga,1,2014/2015
2,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,1:0,38',NaN,Bundesliga,1,2014/2015
9,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",1899 Hoffenheim,FC Augsburg,NaN,2:0,35',NaN,Bundesliga,1,2014/2015
11,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",1899 Hoffenheim,FC Augsburg,NaN,1:0,33',NaN,Bundesliga,1,2014/2015
36,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",Eintracht Frankfurt,SC Freiburg,NaN,1:0,15',NaN,Bundesliga,1,2014/2015
15,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",Hannover 96,FC Schalke 04,NaN,2:1,70',NaN,Bundesliga,1,2014/2015
17,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",Hannover 96,FC Schalke 04,NaN,1:1,67',NaN,Bundesliga,1,2014/2015
20,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",Hannover 96,FC Schalke 04,NaN,0:1,47',NaN,Bundesliga,1,2014/2015
25,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",Hertha BSC,SV Werder Bremen,NaN,2:2,55',NaN,Bundesliga,1,2014/2015


In [7]:
def merge_goal_times(df):
    """
    Fügt die Torzeiten in die gleiche Zeile wie die Tore ein und übernimmt die Daten aus der Spalte 'extratime_raw',
    ohne Daten zu löschen, wenn es mehr Torzeiten als Tore gibt.

    :param df: Der zu bearbeitende DataFrame.
    :return: Der bearbeitete DataFrame.
    """
    # Gruppieren der Daten nach Spielen
    grouped = df.groupby(['matchday_raw', 'hometeam_raw', 'awayteam_raw'])

    # Listen zur Speicherung der bearbeiteten Daten
    merged_data = []

    # Durchlaufen jeder Gruppe (jedes Spiels)
    for _, group in grouped:
        # Extrahieren der Tore, Torzeiten und Extrazeiten
        goals = group['goal_raw'].dropna().tolist()
        times = group['goal_time_raw'].dropna().tolist()
        extratimes = group['extratime_raw'].dropna().tolist()

        # Sicherstellen, dass die Länge der Listen gleich ist
        max_length = max(len(goals), len(times), len(extratimes))
        goals += [None] * (max_length - len(goals))
        times += [None] * (max_length - len(times))
        extratimes += [None] * (max_length - len(extratimes))

        # Erstellen einer neuen DataFrame für das aktuelle Spiel
        game_data = group.iloc[:max_length].copy()
        game_data['goal_raw'] = goals
        game_data['goal_time_raw'] = times
        game_data['extratime_raw'] = extratimes

        # Hinzufügen der bearbeiteten Daten zur Liste
        merged_data.append(game_data)

    # Zusammenführen aller bearbeiteten Spiele zurück in einen DataFrame
    return pd.concat(merged_data)

# Beispiel für die Verwendung der Funktion
# df_example = pd.DataFrame(...)  # Ersetzen Sie dies durch Ihren DataFrame
df_cleaned = merge_goal_times(df_example)
df_cleaned.head()


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,League,Spieltag,Saison
0,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:1,52',no,Bundesliga,1,2014/2015
1,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:0,47',no,Bundesliga,1,2014/2015
2,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,1:0,38',no,Bundesliga,1,2014/2015
9,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",1899 Hoffenheim,FC Augsburg,NaN,2:0,35',no,Bundesliga,1,2014/2015
11,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",1899 Hoffenheim,FC Augsburg,NaN,1:0,33',no,Bundesliga,1,2014/2015


In [8]:
def update_final_score(df):
    """
    Aktualisiert den final_score_raw basierend auf dem spätesten Tor in jedem Spiel.
    Spiele ohne Tore erhalten den final_score_raw '0:0'.

    :param df: Der zu bearbeitende DataFrame.
    :return: Der bearbeitete DataFrame mit aktualisiertem final_score_raw.
    """
    # Gruppieren der Daten nach Spielen
    grouped = df.groupby(['matchday_raw', 'hometeam_raw', 'awayteam_raw'])

    # Liste zur Speicherung der bearbeiteten Daten
    updated_data = []

    # Durchlaufen jeder Gruppe (jedes Spiels)
    for _, group in grouped:
        # Extrahieren der Tore und Torzeiten
        goals = group['goal_raw'].dropna().tolist()
        times = group['goal_time_raw'].dropna().tolist()

        # Ermitteln des finalen Spielstands
        if goals:
            # Das letzte Tor (basierend auf der Zeit) ist der finale Spielstand
            final_score = sorted(zip(times, goals))[-1][1]
        else:
            # Keine Tore gefallen, daher '0:0'
            final_score = '0:0'

        # Aktualisieren des final_score_raw für das aktuelle Spiel
        game_data = group.copy()
        game_data['final_score_raw'] = final_score

        # Hinzufügen der bearbeiteten Daten zur Liste
        updated_data.append(game_data)

    # Zusammenführen aller bearbeiteten Spiele zurück in einen DataFrame
    return pd.concat(updated_data)

# Anwenden der Funktion auf Ihren DataFrame
df_updated = update_final_score(df_cleaned)
df_updated.head()

,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,League,Spieltag,Saison
0,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:1,52',no,Bundesliga,1,2014/2015
1,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:0,47',no,Bundesliga,1,2014/2015
2,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,1:0,38',no,Bundesliga,1,2014/2015
9,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",1899 Hoffenheim,FC Augsburg,2:0,2:0,35',no,Bundesliga,1,2014/2015
11,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",1899 Hoffenheim,FC Augsburg,2:0,1:0,33',no,Bundesliga,1,2014/2015


In [9]:
def remove_empty_goals(df):
    """
    Entfernt alle Einträge, bei denen 'goal_raw' leer ist.

    :param df: Der zu bearbeitende DataFrame.
    :return: Der bearbeitete DataFrame ohne leere 'goal_raw' Einträge.
    """
    # Entfernen von Zeilen, in denen 'goal_raw' leer ist
    df_cleaned = df.dropna(subset=['goal_raw'])
    return df_cleaned

# Anwenden der Funktion auf Ihren DataFrame
df_updated = remove_empty_goals(df_updated)
df_updated.head()


,matchday_raw,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw,League,Spieltag,Saison
0,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:1,52',no,Bundesliga,1,2014/2015
1,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,2:0,47',no,Bundesliga,1,2014/2015
2,"Bundesliga 2014/2015 - 1. Spieltag - Fr., 22.0...",FC Bayern München,VfL Wolfsburg,2:1,1:0,38',no,Bundesliga,1,2014/2015
9,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",1899 Hoffenheim,FC Augsburg,2:0,2:0,35',no,Bundesliga,1,2014/2015
11,"Bundesliga 2014/2015 - 1. Spieltag - Sa., 23.0...",1899 Hoffenheim,FC Augsburg,2:0,1:0,33',no,Bundesliga,1,2014/2015


In [10]:
def rearrange_columns(df):
    """
    Entfernt die Spalte 'matchday_raw' und ändert die Reihenfolge der Spalten.

    :param df: Der zu bearbeitende DataFrame.
    :return: Der bearbeitete DataFrame mit der neuen Spaltenreihenfolge.
    """
    # Entfernen der Spalte 'matchday_raw'
    df = df.drop(columns=['matchday_raw'])

    # Festlegen der neuen Spaltenreihenfolge
    new_order = ['League', 'Saison', 'Spieltag', 'hometeam_raw', 'awayteam_raw',
                 'final_score_raw', 'goal_raw', 'goal_time_raw', 'extratime_raw']

    # Anordnen der Spalten in der neuen Reihenfolge
    df = df[new_order]
    return df

# Anwenden der Funktion auf Ihren DataFrame
df_ord = rearrange_columns(df_updated)
df_ord.head()

,League,Saison,Spieltag,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw
0,Bundesliga,2014/2015,1,FC Bayern München,VfL Wolfsburg,2:1,2:1,52',no
1,Bundesliga,2014/2015,1,FC Bayern München,VfL Wolfsburg,2:1,2:0,47',no
2,Bundesliga,2014/2015,1,FC Bayern München,VfL Wolfsburg,2:1,1:0,38',no
9,Bundesliga,2014/2015,1,1899 Hoffenheim,FC Augsburg,2:0,2:0,35',no
11,Bundesliga,2014/2015,1,1899 Hoffenheim,FC Augsburg,2:0,1:0,33',no


In [14]:
# Assume df is your DataFrame loaded with a larger dataset
# You can replace this with reading from a CSV or any other data source
# For example: df = pd.read_csv('your_data.csv')

# Remove the apostrophe at the end of the strings in 'goal_time_raw'
df_ord['goal_time_raw'] = df_ord['goal_time_raw'].str.replace("'", "")

# The DataFrame df is now updated with the changes
df_ord.head(10)

,League,Saison,Spieltag,hometeam_raw,awayteam_raw,final_score_raw,goal_raw,goal_time_raw,extratime_raw
0,Bundesliga,2014/2015,1,FC Bayern München,VfL Wolfsburg,2:1,2:1,52,no
1,Bundesliga,2014/2015,1,FC Bayern München,VfL Wolfsburg,2:1,2:0,47,no
2,Bundesliga,2014/2015,1,FC Bayern München,VfL Wolfsburg,2:1,1:0,38,no
9,Bundesliga,2014/2015,1,1899 Hoffenheim,FC Augsburg,2:0,2:0,35,no
11,Bundesliga,2014/2015,1,1899 Hoffenheim,FC Augsburg,2:0,1:0,33,no
36,Bundesliga,2014/2015,1,Eintracht Frankfurt,SC Freiburg,1:0,1:0,15,no
15,Bundesliga,2014/2015,1,Hannover 96,FC Schalke 04,2:1,2:1,70,no
17,Bundesliga,2014/2015,1,Hannover 96,FC Schalke 04,2:1,1:1,67,no
20,Bundesliga,2014/2015,1,Hannover 96,FC Schalke 04,2:1,0:1,47,no
25,Bundesliga,2014/2015,1,Hertha BSC,SV Werder Bremen,2:2,2:2,55,no


In [15]:
def save_to_excel(df, path="/Users/youri/VSC Data/Data Analytics/Bundesliga_Bereinigt.xlsx"):
    """
    Speichert den übergebenen DataFrame in einer Excel-Datei im angegebenen Verzeichnis.

    :param df: Der DataFrame, der gespeichert werden soll.
    :param path: Der vollständige Pfad und Name der zu erstellenden Excel-Datei.
    """
    try:
        # Speichern des DataFrame in einer Excel-Datei
        df.to_excel(path, index=False)
        return f"Datei '{path}' wurde erfolgreich gespeichert."
    except Exception as e:
        return f"Ein Fehler ist aufgetreten: {e}"

# Speichern des df_example DataFrame in einer Excel-Datei
save_to_excel(df_ord)


"Datei '/Users/youri/VSC Data/Data Analytics/Bundesliga_Bereinigt.xlsx' wurde erfolgreich gespeichert."

In [ ]:


# Path to the Excel file
excel_path = '/Users/youri/VSC Data/Data Analytics/Fussballdaten/Bundesliga/Bundesliga_Bereinigt.xlsx'  # Update this to your Excel file path

# Read the Excel file
df_ord = pd.read_excel(excel_path)

# Path where you want to save the CSV file
csv_path = '/Users/youri/VSC Data/Data Analytics/Fussballdaten/Bundesliga/Bundesliga_Bereinigt.csv'  # Update this to your desired CSV file path

# Write the DataFrame to a CSV file
df_ord.to_csv(csv_path, index=False)
